# Example CP-APR with PyTorch

In [1]:
import scipy.io as spio
import numpy as np

Load the sample dataset:

In [2]:
X = spio.loadmat('../data/test_data/subs.mat', squeeze_me=True)['values'] - 1
vals = spio.loadmat('../data/test_data/vals_count.mat', squeeze_me=True)['values']

We can also set the initial values of the latent factors:

In [3]:
# Initial factor values
M_init = dict()
dim = 0
for key, values in spio.loadmat('../data/test_data/minit.mat', squeeze_me=True).items():
    if 'init_f' in key:
        M_init[str(dim)] = values
        dim += 1

## PyTorch on GPU with initial M values

Initilize:

In [4]:
from pyCP_APR import CP_APR

cp_apr = CP_APR(n_iters=100, verbose=10, method='torch', device='gpu', device_num=0)

Using TITAN RTX


Take the CP-APR decomposition:

In [5]:
result = cp_apr.fit(coords=X, values=vals, rank=2, Minit=M_init)

/opt/conda/envs/pyCP_APR/lib/python3.8/site-packages/pyCP_APR-1.0.0-py3.8.egg/pyCP_APR/torch_cp/CP_APR_Torch.py:422: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.epsilon = tr.tensor(self.epsilon).to(self.device)


CP-APR (MU):
Iter=1, Inner Iter=30, KKT Violation=0.397068, obj=-5012.285626, nViolations=0
Iter=11, Inner Iter=21, KKT Violation=0.137084, obj=-3147.301615, nViolations=0
Iter=21, Inner Iter=19, KKT Violation=0.000397, obj=-3145.330122, nViolations=0
Iter=31, Inner Iter=12, KKT Violation=0.000108, obj=-3145.325967, nViolations=0
Iter=41, Inner Iter=12, KKT Violation=0.000108, obj=-3145.325967, nViolations=0
Iter=51, Inner Iter=12, KKT Violation=0.000108, obj=-3145.325967, nViolations=0
Iter=61, Inner Iter=12, KKT Violation=0.000217, obj=-3145.325966, nViolations=0
Iter=71, Inner Iter=12, KKT Violation=0.000438, obj=-3145.325963, nViolations=0
Iter=81, Inner Iter=12, KKT Violation=0.000672, obj=-3145.325959, nViolations=0
Iter=91, Inner Iter=13, KKT Violation=0.001141, obj=-3145.325950, nViolations=0
 Final log-likelihood = -3145.325933
 Final least squares fit = 0.017369
 Final KKT violation = 0.001531
 Total inner iterations = 1462
 Total execution time = 1.0486 seconds
Converting th

In [6]:
result

{'Factors': {'0': array([[5.88838457e-51, 2.13058370e-01],
         [3.23364716e-04, 1.34610100e-01],
         [2.05013230e-01, 7.12928005e-37],
         [1.48424405e-01, 0.00000000e+00],
         [9.76200219e-02, 1.48477484e-01],
         [2.51566211e-02, 2.06908903e-01],
         [1.43573934e-01, 3.34319439e-88],
         [2.61925420e-01, 4.76257924e-33],
         [9.37106506e-02, 1.87295857e-01],
         [2.42523537e-02, 1.09649287e-01]]),
  '1': array([[2.31775360e-241, 5.03672967e-002],
         [7.79309622e-002, 1.00144467e-137],
         [0.00000000e+000, 7.84481789e-002],
         [1.23105143e-001, 9.77480876e-002],
         [3.30736653e-002, 5.64828345e-002],
         [1.56285154e-078, 9.36029407e-003],
         [4.85047483e-002, 0.00000000e+000],
         [1.38753783e-001, 1.68960429e-002],
         [6.85533776e-002, 8.11225486e-002],
         [9.22432344e-003, 3.54237911e-002],
         [8.73085469e-002, 0.00000000e+000],
         [1.71383444e-002, 6.32934171e-002],
       

## PyTorch with Sparse Torch Tensor as X on GPU:

In [7]:
import torch

i = torch.LongTensor([[0, 1, 1],
                      [2, 0, 2],
                      [2, 0, 1]])

v = torch.FloatTensor([3, 4, 5])
X = torch.sparse.FloatTensor(i, v, torch.Size([4,4,4]))
X

tensor(indices=tensor([[0, 1, 1],
                       [2, 0, 2],
                       [2, 0, 1]]),
       values=tensor([3., 4., 5.]),
       device='cpu', size=(4, 4, 4), nnz=3, dtype=torch.float32,
       layout=torch.sparse_coo)

In [8]:
cp_apr = CP_APR(n_iters=100, verbose=10, device='gpu')
result = cp_apr.fit(tensor=X, rank=30)

Using TITAN RTX
CP-APR (MU):
Iter=1, Inner Iter=30, KKT Violation=0.425532, obj=4.887921, nViolations=0
Exiting because all subproblems reached KKT tol.
 Final log-likelihood = 4.888204
 Final least squares fit = 0.999995
 Final KKT violation = 0.000007
 Total inner iterations = 37
 Total execution time = 0.2338 seconds
Converting the latent factors to Numpy arrays.
